- #### Transformaciones
    - ##### join
    - ##### UDF

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

In [ ]:
def read_csv(path):
    return spark.read\
        .option("header","true")\
        .option("delimiter",",")\
        .option("inferSchema","false")\
        .csv(path)

base_path = "../../resources/data/csv/"
clients_df = read_csv(base_path + "clients.csv")
contracts_df = read_csv(base_path + "contracts.csv")
products_df = read_csv(base_path + "products.csv")

clients_df.show()
contracts_df.show()
products_df.show()

In [ ]:
# Joins

# inner -> Mantiene información de ambas tablas (columnas) para los registros (filas) coincidentes
# outer -> Mantiene información de ambas tablas (columnas y filas) para los registros coincidentes y no-coincidentes
# left -> Mantiene columnas de ambas tablas y filas únicamente de la tabla izquierda, elimina filas no coincidentes de la tabla derecha
# right -> Mantiene columnas de ambas tablas y filas únicamente de la tabla derecha, elimina filas no coincidentes de la tabla izquierda
# left_semi -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros coincidentes
# left_anti -> Mantiene filas y columnas únicamente de la tabla izquierda para los registros no-coincidentes

# cross

In [ ]:
import pyspark.sql.functions as f

clients_tmp_df = clients_df.filter((f.col("edad") >= 40) & (f.col("edad") <= 50))
contracts_tmp_df = contracts_df.filter(f.col("activo") == "false")\
    .withColumnRenamed("cod_titular", "cod_client")

clients_tmp_df.show()
contracts_tmp_df.show()

# clients_tmp_df.crossJoin(contracts_tmp_df).show() ## WARNING

typw_join = "full"   # inner, outer, left, right, left_semi, left_anti

join_df = clients_tmp_df.join(contracts_tmp_df, ["cod_client"], typw_join)
join_df.show()

In [ ]:
# UDF - User Defined Function - WARNING
import pyspark.sql.types as t

def upperCase(value):
    if value is None:
        return ""
    else:
        return value.upper()

def len_concat(item_1, item_2):
    if item_1 is None:
        item_1 = ""
    if item_2 is None:
        item_2 = ""
    return len(item_1 + item_2)

upper_udf = f.udf(upperCase, t.StringType())

len_concat_udf = f.udf(len_concat, t.LongType())

join_df.select(
    *join_df.columns,
    upper_udf(f.col("nombre")).alias("nombre_mayus"),
    len_concat_udf(f.col("nombre"), f.col("provincia")).alias("len_concat")
).show()

In [ ]:
from pyspark.sql import Row

tmp_df = spark.createDataFrame([
    Row(1,None),
    Row(2,float("nan")),
    Row(3,3.2),
    Row(4,float("nan"))], ["id", "number"])

In [ ]:
tmp_df.show()

def is_null_or_nan(value):
    if (value!=value) | (value is None):
        return True
    else:
        return False

is_null_or_nan_udf = f.udf(is_null_or_nan, t.BooleanType())

tmp_df.select(
    *tmp_df.columns,
    is_null_or_nan_udf(f.col("number")).alias("null_nan")
).show()